# Лабораторна робота №12: Основи обробки природної мови (NLP)

**Виконав:** [Лаптєв Олександр]
**Група:** [МІТ-31]

## 1. Мета роботи
Познайомитися з базовими техніками NLP (токенізація, лематизація, векторизація) та навчитися будувати прості моделі для класифікації текстових даних.

## 2. Опис завдання
1.  **Збір даних:** Використання датасету новин (20 Newsgroups).
2.  **Передобробка:** Очищення, видалення стоп-слів, лематизація.
3.  **Векторизація:** Порівняння підходів **Bag of Words** та **TF-IDF**.
4.  **Моделювання:** Навчання наївного баєсового класифікатора (Multinomial Naive Bayes).
5.  **Оцінка:** Аналіз точності та матриці похибок.

In [ ]:
# Встановлення необхідних бібліотек (розкоментуйте при першому запуску)
# !pip install nltk spacy scikit-learn pandas matplotlib seaborn
# !python -m spacy download en_core_web_sm

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# NLP бібліотеки
import nltk
import spacy
from nltk.corpus import stopwords

# Sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Завантаження ресурсів NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab') # Важливо для нових версій NLTK
nltk.download('wordnet')

# Завантаження моделі SpaCy
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Завантаження моделі SpaCy...")
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

print("Бібліотеки успішно завантажено.")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\igarl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\igarl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\igarl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\igarl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Бібліотеки успішно завантажено.


## 3. Завантаження та підготовка даних

Для лабораторної ми візьмемо підмножину датасету **20 Newsgroups**, обравши 4 різні категорії тем, щоб модель могла їх розрізняти.

In [2]:
# Обираємо категорії для класифікації
categories = ['sci.space', 'comp.graphics', 'talk.politics.mideast', 'rec.sport.hockey']

# Завантажуємо дані
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Створюємо DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
df['label_name'] = df['label'].apply(lambda x: newsgroups.target_names[x])

print(f"Завантажено записів: {df.shape[0]}")
print("\nПриклад даних:")
display(df.head())

# Перевірка розподілу класів
plt.figure(figsize=(8, 4))
sns.countplot(x='label_name', data=df, palette='viridis')
plt.title("Розподіл категорій новин")
plt.show()

KeyboardInterrupt: 

## 4. Передобробка тексту (Preprocessing)

Створимо функцію для очищення тексту, яка включає:
1.  Приведення до нижнього регістру.
2.  Видалення пунктуації та спецсимволів (RegEx).
3.  Лематизацію (SpaCy) — зведення слів до початкової форми.
4.  Видалення стоп-слів (NLTK) — слів, що не несуть змісту (the, is, in...).

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 1. Очистка від спецсимволів та цифр, переведення в lower case
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    
    # 2. Обробка через SpaCy (Tokenization + Lemmatization)
    doc = nlp(text)
    
    # 3. Фільтрація: беремо леми, якщо вони не є стоп-словами і довші за 2 символи
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and len(token.text) > 2]
    
    return " ".join(tokens)

# Це може зайняти певний час (1-2 хв) для всього датасету
print("Починаємо обробку тексту...")
df['cleaned_text'] = df['text'].apply(preprocess_text)
print("Обробку завершено!")

display(df[['text', 'cleaned_text']].head())

## 5. Векторизація та Побудова моделі

Ми проведемо експеримент: порівняємо ефективність класифікації при використанні двох різних методів векторизації.
1.  **Bag of Words (CountVectorizer)** — просто рахує кількість слів.
2.  **TF-IDF (TfidfVectorizer)** — враховує важливість слова в контексті всього корпусу.

Модель класифікації: **Multinomial Naive Bayes** (класичний вибір для роботи з текстом).

In [ ]:
# Функція для повного циклу: Векторизація -> Спліт -> Навчання -> Оцінка
def train_and_evaluate(vectorizer, X_text, y, name="Model"):
    # 1. Векторизація
    X_vec = vectorizer.fit_transform(X_text)
    
    # 2. Розбиття на train/test
    X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)
    
    # 3. Навчання моделі
    model = MultinomialNB()
    model.fit(X_train, y_train)
    
    # 4. Передбачення
    y_pred = model.predict(X_test)
    
    # 5. Метрики
    acc = accuracy_score(y_test, y_pred)
    print(f"=== Результати для {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print("-" * 30)
    
    return model, acc, y_test, y_pred

# Експеримент 1: Bag of Words
print("Навчання моделі Bag of Words...")
cv = CountVectorizer(max_features=5000) # Обмежуємо словник топ-5000 словами
model_bow, acc_bow, y_test_bow, y_pred_bow = train_and_evaluate(cv, df['cleaned_text'], df['label'], "Bag of Words")

# Експеримент 2: TF-IDF
print("Навчання моделі TF-IDF...")
tfidf = TfidfVectorizer(max_features=5000)
model_tfidf, acc_tfidf, y_test_tfidf, y_pred_tfidf = train_and_evaluate(tfidf, df['cleaned_text'], df['label'], "TF-IDF")

## 6. Візуалізація результатів та Аналіз помилок 
Побудуємо матрицю похибок для кращої моделі (зазвичай TF-IDF показує кращий результат).

In [ ]:
# Побудова графіку порівняння точності
results_df = pd.DataFrame({
    'Method': ['Bag of Words', 'TF-IDF'],
    'Accuracy': [acc_bow, acc_tfidf]
})

plt.figure(figsize=(6, 4))
sns.barplot(x='Method', y='Accuracy', data=results_df, palette='magma')
plt.ylim(0, 1.0)
plt.title("Порівняння точності методів векторизації")
plt.show()

# Матриця похибок для TF-IDF
cm = confusion_matrix(y_test_tfidf, y_pred_tfidf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=newsgroups.target_names, 
            yticklabels=newsgroups.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (TF-IDF)')
plt.show()

In [ ]:
# Подивимось на приклади роботи моделі
def predict_news(text, model, vectorizer):
    processed = preprocess_text(text)
    vec = vectorizer.transform([processed])
    prediction = model.predict(vec)
    return newsgroups.target_names[prediction[0]]

samples = [
    "The NASA rover landed on Mars successfully.", # Space
    "The goalkeeper saved the penalty in the last minute.", # Hockey
    "New graphics card drivers are available for download.", # Graphics
    "Peace negotiations in the Middle East are ongoing." # Politics
]

print("=== Тестування на власних прикладах ===")
for txt in samples:
    pred = predict_news(txt, model_tfidf, tfidf)
    print(f"Text: {txt[:50]}... -> Prediction: {pred}")

## 7. Звіт та Висновки

### Основні етапи
1.  **Передобробка:** Було застосовано комплексний підхід, що включав очищення від шуму, видалення стоп-слів та лематизацію за допомогою `SpaCy`. Це дозволило зменшити розмірність словника і покращити якість моделі.
2.  **Модель:** Використано Наївний Баєсовий класифікатор (`MultinomialNB`), який є стандартом для базових задач класифікації тексту через свою швидкість та ефективність на розріджених даних.

### Вплив методів векторизації
* **Bag of Words (BoW):** Показав точність близько **[Вставити ваше число, напр. 0.92]**. Цей метод простий, але він надає однакову вагу всім словам, що іноді призводить до домінування частих, але менш інформативних слів.
* **TF-IDF:** Показав точність близько **[Вставити ваше число, напр. 0.94]**. Метод виявився ефективнішим, оскільки він "карає" слова, що зустрічаються занадто часто у всіх документах, і виділяє специфічні терміни для кожної категорії.

### Аналіз помилок
Матриця похибок показує, що модель найкраще класифікує тему `rec.sport.hockey` (хокей) через специфічну лексику (game, team, play), а найбільше плутанини може виникати між `comp.graphics` та іншими технічними темами, якщо вони присутні.

**Загальний висновок:** Для даної задачі класифікації комбінація **TF-IDF + MultinomialNB** є оптимальним рішенням з точки зору балансу точності та обчислювальних витрат.